In [1]:
import pandas as pd
import bqplot
import us
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import cartopy
import ipywidgets
import matplotlib.cm as cm
%matplotlib inline

In [2]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link
from bqplot import (
    Figure, Map, Mercator, Orthographic, ColorScale, ColorAxis,
    AlbersUSA, topo_load, Tooltip
)
from ipywidgets import ToggleButtons, VBox, HTML, Dropdown, HBox

In [3]:
#start code (instructor provided)
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]
fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
fips_count = ufo.groupby("fips")["duration_seconds"].count()

## The following are codes for plotting the total number of sightings/duration_seconds in whichever state is highlighted with a function of year

In [4]:
#[code for plot] get the intended columns and extract year from the date column
basic_info = ufo[['state','date','duration_seconds']]
basic_info['year']=pd.DatetimeIndex(basic_info['date']).year
basic_data = basic_info[['state','year','duration_seconds']]

#[code for plot] transform the state to fips  [code for plot]
abbr_to_fits = us.states.mapping('abbr', 'fips')
basic_data["fips"] = basic_data["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
ppdata = basic_data[['fips','year','duration_seconds']]

#[code for plot] use the groupby function for fips  [code for plot]
duration_data = ppdata.groupby(['fips','year']).sum()
sightings_data = ppdata.groupby(['fips','year']).count()
plotdata = duration_data
plotdata['total_sightings']=sightings_data['duration_seconds']

#[code for plot] quickview of the data for plotting
plotdata.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


duration_seconds  total_sightings
fips year                                   
-1   1906           10800.0                1
     1916              60.0                1
     1936            1200.0                1
     1943            1510.0                4
     1944            1380.0                3

## Now it is the function of interval plot for 'duration_seconds'

In [7]:
def durationplot(fips):
## First we define a Figure
    testdata=plotdata.loc[fips]
    dt_x_fast = LinearScale()
    lin_y = LinearScale()

    dates_actual = testdata.index.values
    prices = testdata.duration_seconds

    x_ax = Axis(label='Year', scale=dt_x_fast)
    x_ay = Axis(label='Duration_seconds', scale=lin_y, orientation='vertical')
    lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])
    
    return lc

## Now it is the function of interval plot for 'total_sightings'

In [8]:
def sightings(fips):
## First we define a Figure
    testdata=plotdata.loc[fips]
    dt_x_fast = LinearScale()
    lin_y = LinearScale()

    dates_actual = testdata.index.values
    prices = testdata.total_sightings

    x_ax = Axis(label='Year', scale=dt_x_fast)
    x_ay = Axis(label='Total_sightings', scale=lin_y, orientation='vertical')
    lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])

    return lc

## The following are codes for map plot

In [9]:
#[code for map]
duration_seconds = ufo.groupby(["fips"])["duration_seconds"].sum()
total_sightings = ufo.groupby(["fips"])["report_date"].count()
#[code for map]
df = pd.DataFrame([duration_seconds, total_sightings])
df.rename({'report_date': 'total_sightings'}, inplace=True)
ufo_vals = df.transpose()
ufo_vals["Normalized total sightings"] = [None , 4863300, 741894, 6931071, 2988248, 39250017, 5540545, 3576452, 952065, 681170, 20612439, 10310371, 1428557, 1683140, 12801539, 6633053, 3134693, 2907289, 4436974, 4681666, 1331479, 6016447, 6811779, 9928300, 5519952, 2988726, 6093000, 1042520, 1907116, 2940058, 1334795, 8944469, 2081015, 19745289, 10146788, 757952, 11614373, 3923561, 4093465, 12784227, 1056426, 4961119, 865454, 6651194, 27862596, 3051217, 624594, 8411808, 7288000, 1831102, 5778708, 585501, None]
#[code for map]
states = cartopy.io.shapereader.natural_earth(resolution='110m', category='cultural',
                                    name='admin_1_states_provinces_lakes_shp')
reader =cartopy.io.shapereader.Reader(states)
all_states = list(reader.records())

In [12]:
@ipywidgets.interact(field = ['total_sightings', 'duration_seconds','Normalized total sightings'] )
def make_plot(field):
    for fips in ufo_vals:
        n = np.log10(ufo_vals.loc[:, field])
        norm = (ufo_vals['total_sightings'])/(ufo_vals['Normalized total sightings'])
    def_tt = Tooltip(fields=['name', 'color'], labels=['State', field],formats = [])    
    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                      'color': bqplot.ColorScale(colors=["red", "blue"])},
              'color': n.to_dict(),
                 'interactions':{'click': 'select', 'hover': 'tooltip'}}
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'), tooltip=def_tt,
        **map_styles) 
    states_map.interactions = {'click': 'select', 'hover': 'tooltip'}
    map_fig = bqplot.Figure(marks=[states_map], title='USA',fig_margin={'top':0,'bottom':20,'left':10,'right':10})
    
    if field == 'total_sightings':
        testdata=plotdata.loc[1]
        print('Please select a state, the default state is Alabama.')
        dt_x_fast = LinearScale()
        lin_y = LinearScale()

        dates_actual = testdata.index.values
        prices = testdata.total_sightings

        x_ax = Axis(label='Year', scale=dt_x_fast)
        x_ay = Axis(label='Total_sightings', scale=lin_y, orientation='vertical')
        lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])
        intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[lc])
        md = {}
        def fast_interval_change_callback(change):
            ind = pd.to_datetime(change.new)
            ind = change.new
            tot = testdata.loc[ind[0]:ind[1]]["total_sightings"].sum()
            db_fast.value = 'The selected period is ' + str(change.new).split(' ')[0][1:5] + ' to '+ str(change.new).split(' ')[1][0:4] + ' total: ' + str(tot)
        intsel_fast.observe(fast_interval_change_callback, names=['selected'])
        db_fast = HTML()
        db_fast.value = 'The selected period is ' + str(intsel_fast.selected)

        fig_fast_intsel = Figure(marks=[lc], axes=[x_ax, x_ay], title='Total_sightings of selected state',
                         interaction=intsel_fast,fig_margin={'top':0,'bottom':20,'left':50,'right':180}) 
        #This is where we assign the interaction to this particular Figure

        def change_selected(event):
            try: 
                lc.y=sightings(event.new[-1]).y
                
            except:
                t=1+1
        states_map.observe(change_selected, "selected")
        display(HBox([map_fig,VBox([db_fast, fig_fast_intsel])]))
    
    elif field == 'duration_seconds':
        testdata=plotdata.loc[1]
        print('Please select a state, the default state is Alabama.')
        dt_x_fast = LinearScale()
        lin_y = LinearScale()

        dates_actual = testdata.index.values
        prices = testdata.duration_seconds

        x_ax = Axis(label='Year', scale=dt_x_fast)
        x_ay = Axis(label='Duration_seconds', scale=lin_y, orientation='vertical')
        lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])
        intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[lc])
        md = {}
        def fast_interval_change_callback(change):
            ind = pd.to_datetime(change.new)
            ind = change.new
            tot = testdata.loc[ind[0]:ind[1]]["duration_seconds"].sum()
            db_fast.value = 'The selected period is ' + str(change.new).split(' ')[0][1:5] + ' to '+ str(change.new).split(' ')[1][0:4] + ' total: ' + str(tot)
        intsel_fast.observe(fast_interval_change_callback, names=['selected'])
        db_fast = HTML()
        db_fast.value = 'The selected period is ' + str(intsel_fast.selected)

        fig_fast_intsel = Figure(marks=[lc], axes=[x_ax, x_ay], title='Duration_Seconds of selected state',
                         interaction=intsel_fast,fig_margin={'top':10,'bottom':20,'left':50,'right':180}) 
        #This is where we assign the interaction to this particular Figure
        

        def change_selected(event):
            try:
                lc.y=durationplot(event.new[-1]).y
            except:
                t=1+1
        states_map.observe(change_selected, "selected")
        display(HBox([map_fig,VBox([db_fast, fig_fast_intsel])]))
        
    elif field == 'Normalized total sightings':
        testdata=plotdata.loc[1]
        print('Please select a state, the default state is Alabama.')
        dt_x_fast = LinearScale()
        lin_y = LinearScale()

        dates_actual = testdata.index.values
        prices = testdata.total_sightings

        x_ax = Axis(label='Year', scale=dt_x_fast)
        x_ay = Axis(label='Total_sightings', scale=lin_y, orientation='vertical')
        lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])
        intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[lc])
        md = {}
        def fast_interval_change_callback(change):
            ind = pd.to_datetime(change.new)
            ind = change.new
            tot = testdata.loc[ind[0]:ind[1]]["total_sightings"].sum()
            db_fast.value = 'The selected period is ' + str(change.new).split(' ')[0][1:5] + ' to '+ str(change.new).split(' ')[1][0:4] + ' total: ' + str(tot)            
        intsel_fast.observe(fast_interval_change_callback, names=['selected'])
        db_fast = HTML()
        db_fast.value = 'The selected period is ' + str(intsel_fast.selected)
        fig_fast_intsel = Figure(marks=[lc], axes=[x_ax, x_ay], title='Total_sightings of selected state',
                         interaction=intsel_fast,fig_margin={'top':0,'bottom':20,'left':45,'right':180}) 
        #This is where we assign the interaction to this particular Figure
        def change_selected(event):
            try:
                lc.y=sightings(event.new[-1]).y                
            except:
                t = 1+1
        states_map.observe(change_selected, "selected")
        display(HBox([map_fig,VBox([db_fast, fig_fast_intsel])]))


interactive(children=(Dropdown(description='field', options=('total_sightings', 'duration_seconds', 'Normalized total sightings'), value='total_sightings'), Output()), _dom_classes=('widget-interact',))